In [1]:
#1、tf.get_variable()一般配合tf.variable_scope和reuse一起操作。
#   同一个scope，相同名字的变量需要resue才能使用
#2、tf.Variable()每次都会生成一个新的变量，故而即使重名也不会报错
#3、tf.Variable()必须给定初始化值
#   而tf.get_variable()则必须给定名字和大小

In [2]:
#几种初始化方式：
#1、常量初始化
#   initializer=tf.constant_initializer(2)
#   initializer=tf.constant_initializer(np.arange(6).reshape(2,3))
#2、正态分布初始化
#   tf.random_normal(shape=[2,3], mean=0, stddev=1)
#3、输入输出都服从均匀分布的初始化
#   initializer = initializers.xavier_initializer()     # from tensorflow.contrib.layers.python.layers import initializers
#   initializer = tf.contrib.layers.xavier_initializer()
#4、服从均匀分布，同时可以控制取值范围
#   embeddings = tf.Variable(tf.random_uniform([row, column], -1.0, 1.0)) #随机初始化shape=[row, column]的，取值范围[-1,1]之间服从均匀分布的矩阵

In [3]:
import tensorflow as tf
import numpy as np

D:\Anaconda\Initial\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
#最精简的初始化
a1 = tf.Variable(1)
a2 = tf.get_variable("a2",shape=[1])

In [5]:
#稍微细致一点的初始化
a1_2 = tf.Variable(initial_value=np.arange(9).reshape(3,3),name="a1_2",dtype=tf.int32)
a1_3 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a1_3')

a2_2 = tf.get_variable(name='a2_2', shape=[1], initializer=tf.constant_initializer(2),dtype=tf.int32)
a2_3 = tf.get_variable(name="a2_3",shape=[2,3], initializer=tf.constant_initializer(np.arange(6).reshape(2,3)))
a2_4 = tf.get_variable(name="a2_4",initializer = tf.constant(3,shape=[3])) #shape也可以放在initializer里面进行控制

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(a1.name,end=" ")
    print(a2.name)
    print(a1_2.name,end=" ")
    print(a1_3.name)
    print(a2_2.name)
    print(a2_3.name)
    print("-"*20)
    print(sess.run([a1,a1_2,a2,a2_2,a1_3,a2_3]))
    print(sess.run(a2_4))
#     print(sess.run(a1))
#     print(sess.run(a1_2))
#     print(sess.run(a2))
#     print(sess.run(a2_2))
#     print(sess.run(a1_3))
#     print(sess.run(a2_3))

Variable:0 a2:0
a1_2:0 a1_3:0
a2_2:0
a2_3:0
--------------------
[1, array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]]), array([0.897293], dtype=float32), array([2]), array([[ 2.6300566 ,  1.0007509 ,  0.4158718 ],
       [ 2.6254466 , -0.2537121 , -0.50091326]], dtype=float32), array([[0., 1., 2.],
       [3., 4., 5.]], dtype=float32)]
[3 3 3]


In [7]:
#关于reuse
#1、首先，其目的是为了节省空间，重用变量
#2、如果是新定义，则reuse必须为False，默认值也是False，如果是重用，则必须是True。 
#故而False代表要定义一个新变量，True代表要重用一个旧变量。因此，False时重用旧变量会报错，True是创建新变量会报错。
with tf.variable_scope("one"):
    a = tf.get_variable("v", [1]) #a.name == "one/v:0"

#在没有reuse的情况下，直接get会报错，因为reuse为false，相当于要创建，但是已经存在了，创建即冲突了
# with tf.variable_scope("one"):
#     b = tf.get_variable("v", [1]) #创建两个名字一样的变量会报错 ValueError: Variable one/v already exists 

with tf.variable_scope("one", reuse = True): #注意reuse的作用。
    c = tf.get_variable("v", [1]) #c.name == "one/v:0" 成功共享，因为设置了reuse
    
#或者在使用之前声明要reuse，这样能精确一些范围
with tf.variable_scope("one"): #注意reuse的作用。
    tf.get_variable_scope().reuse_variables()
    d = tf.get_variable("v", [1]) #c.name == "one/v:0" 成功共享，因为设置了reuse
    
print(a==c) #Assertion is true, they refer to the same object.
print(d is a)

True
True


In [8]:
#tf.Variable每次都会生成一个新的变量，故而即使重名也不会报错
a2 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a2')
a4 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a2')
print(a2 == a4)

False


In [9]:
#补充：
#作用域中的reuse默认是False，调用函数reuse_variables()可设置为True。
#一旦设置为True，就不能返回到False，并且该作用域的子空间reuse都是True
#如果不想重用变量，那么可以退回到上层作用域，相当于exit当前作用域

In [10]:
with tf.variable_scope("root"):
    assert tf.get_variable_scope().reuse == False
    with tf.variable_scope("sub"):
        assert tf.get_variable_scope().reuse == False
    with tf.variable_scope("sub",reuse=True):
        assert tf.get_variable_scope().reuse == True
        with tf.variable_scope("sub_sub"):
            assert tf.get_variable_scope().reuse == True
            #d = tf.get_variable("new", [1]) 此时会报错，因为new压根没有定义过，现在reuse为True表示是重用，故而找不到则会报错
    assert tf.get_variable_scope().reuse == False

In [11]:
#那么实际使用中
#要想重复使用某些变量，第一次因为是创建，故而需要reuse=False，以后因为是复用，故而需要reuse=True
#方法一，放在循环里,同时外面加一个reuse变量来判断，比如是在测试时，哪怕是第0次，也设置为True
#for i in range(iter_num):
#    with tf.variable_scope("name",reuse=True if (reuse or j > 0) else False):
#        w = tf.get_variable("filterW",shape=[H, W, C,N],initializer=tf.contrib.layers.xavier_initializer())
#方法二，直接使用tf.AUTO_REUSE
#with tf.variable_scope(name_or_scope='', reuse=tf.AUTO_REUSE):